In [187]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd

In [188]:
driver = webdriver.Chrome(r"C:\Users\hp\chromedriver.exe")

In [189]:
url="https://www.volunteermbc.org/Listings.php?ListType=VolunteerPositionsAll&MenuItemID=1&SetSessionVars=1&http_user_agent=Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F83.0.4103.116+Safari%2F537.36&remote_addr=162.158.74.195&SESSIONID=qfipe3jlqrkooq47bd6apnlqed&age=19-24&Duration=&Availability%5B%5D=%271%27&Availability%5B%5D=%272%27&Availability%5B%5D=%273%27&Availability%5B%5D=%274%27&Availability%5B%5D=%275%27&Availability%5B%5D=%276%27&GeographicalLocation=&AreasOfInterest=&keyword=COVID-19&vol_name=1&Action=Search&Transportation=0&Accessibility=&Agency_Number=&SetSessionVars=1&http_user_agent=Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F83.0.4103.116+Safari%2F537.36&remote_addr=162.158.74.195&SESSIONID=qfipe3jlqrkooq47bd6apnlqed&age=&Duration=&Availability=&Availability%5B%5D=%271%27&Availability%5B%5D=%272%27&Availability%5B%5D=%273%27&Availability%5B%5D=%274%27&Availability%5B%5D=%275%27&Availability%5B%5D=%276%27&GeographicalLocation=&AreasOfInterest=&keyword=COVID-19&vol_name=1&Action=Search&Transportation=0&Accessibility=&Agency_Number="
driver.get(url)

In [190]:
web="https://www.volunteermbc.org/"

In [191]:
pgLnk=[url]
pglink = soup.find_all('a', attrs={'class':'pgLnk pgLnkNum'})
for l in pglink:
    pgLnk.append(web+l.get('href'))

Get main info

In [192]:
Position=[]
Location=[]
Organization=[]
Websites=[]
Description=[]

In [193]:
def get_main_info(mlist,Position, Organization, Location):
    for i in range(len(mlist)):
        pos = mlist[i].find('div',attrs={"class":"cell1"})
        Position.append(pos.get_text())
        org = mlist[i].find_all('div',attrs={"class":"cell2"})
        Organization.append(org[0].get_text().split(":")[1])
        Location.append(org[1].get_text().split(":")[1])
    return Position, Organization, Location

In [194]:
def get_websites(soup):
    webs = soup.find_all('span', attrs={"class": "customLink"})
    for i in range(len(webs)):
        w = webs[i].find_all("a")
        for link in w:
            Websites.append(web+link.get("href"))
    return Websites

In [195]:
def get_description(Websites):
    Description=[]
    for link in Websites:
        Content=[]
        html_content = requests.get(link).text
        soup = BeautifulSoup(html_content, "lxml")
        des=soup.find_all('tr')
        idx_des = [i for i in range(len(des)) if des[i].find(string='Description') != None]
        Content.append(des[idx_des[-1]].get_text())
        idx_des = [i for i in range(len(des)) if des[i].find(string='Goal') != None]
        Content.append(des[idx_des[-1]].get_text())
        idx_des = [i for i in range(len(des)) if des[i].find(string='Benefits') != None]
        Content.append(des[idx_des[-1]].get_text())
        idx_des = [i for i in range(len(des)) if des[i].find(string='Position Qualifications') != None]
        Content.append(des[idx_des[-1]].get_text())
        idx_des = [i for i in range(len(des)) if des[i].find(string='Time Commitment') != None]
        Content.append(des[idx_des[-1]].get_text())
        idx_des = [i for i in range(len(des)) if des[i].find(string='Training') != None]
        Content.append(des[idx_des[-1]].get_text())
        Description.append(Content)
    return Description

In [196]:
for i in range(len(pgLnk)):
    html_content = requests.get(pgLnk[i]).text
    soup = BeautifulSoup(html_content, "lxml")
    mainlist = soup.find_all('div',attrs={'class': 'MainList'})
    Position, Organization, Location = get_main_info(mainlist, Position, Organization, Location)
    Websites = get_websites(soup)
    Description = get_description(Websites)

Location:
<br> M: Mississauga
<br> B: Brampton
<br> C: Caledon
<br> V: Virtual

In [197]:
df = pd.DataFrame({'Position':Position,'Location':Location, 
                 'Organization':Organization,
                  "Websites": Websites,
                  "Description": Description}) 

In [198]:
df.head()

,Position,Location,Organization,Websites,Description
0,Telephone Reassurance Program (ACTIVE during t...,M/B/C/V,"Victorian Order of Nurses, Peel Region, Comm...",https://www.volunteermbc.org/report.php?ListTy...,[\n\n\n\nDescriptionBrighten the day of an iso...
1,COVID-19 Community Response DRIVERS (on call),M/B/C/V,Volunteer MBC,https://www.volunteermbc.org/report.php?ListTy...,[\n\n\n\nDescription> Volunteers are 'on-call'...
2,Board of Directors - Member (Please apply duri...,M/B/C/V,North Peel & Dufferin Community Legal Services,https://www.volunteermbc.org/report.php?ListTy...,[\n\n\n\nDescriptionWe are particularly intere...
3,Meals On Wheels Volunteer Driver - solo or in ...,Mississauga,"Victorian Order of Nurses, Peel Region, Comm...",https://www.volunteermbc.org/report.php?ListTy...,[\n\n\n\nDescriptionThis position is ideal for...
4,Marketing Career Mentor (VIRTUAL during COVID-...,M/B/C/V,Sheridan College Community Employment Services,https://www.volunteermbc.org/report.php?ListTy...,[\n\n\n\nDescriptionMentors are paired with an...


In [199]:
df.to_csv('MBC_volunteer.csv', index=False, encoding='utf-8_sig')